# Modèles génératifs markoviens

Dans ce TP, nous allons étudier la possibilité de générer du texte en langage naturel via une modélisation statistique.

Les langages naturels (le français, l'anglais etc) sont à la fois fortement structurés et très riches :
* On peut écrire une infinité de phrases correctes différentes
* L'immense majorité des suites de mots ne forme pas des phrases

La structuration grammaticale du langage entraîne que la distribution de fréquence marginale d'un mot est très différente de celle conditionnée par son contexte (*i.e.*, étant donnés les mots qui précèdent) : 

$$ P(X_i) \neq P(X_i|X_{i-1},\dots,X_1) $$

Or on peut raisonnablement considérer qu'un page particulière d'un romab particulier, qui contient entre 250 et 280 mots, identifie de manière unique le roman en question. Cela implique que le mot qui suit immédiatement cette page est determiné de manière unique dans l'ensemble de la production litteraire de l'humanité. Il en résulte que pour une page originale grammaticalement correcte, aucune donnée statistique n'est disponible pour determiner la probabilité qu'un mot particulier ne vienne ensuite. 

Notre rêve de pouvoir simplement modéliser la distribution de probabilité jointe $P(X_1,\dots,X_100000)$ de tous les livres de 100000 mots possibles et de simplement en tirer un au hasard s'évanouit définitivement...

Notons également qu'une telle distribution possède $n^100000$ dimensions où $n$ est le nombre de mots valides de la langue. Notre cause est donc désespérée

Nous allons donc devoir considérer un **contexte limité**. 

**Définition - Chaîne de Markov:** Une chaîne de Markov est une suite de variables aléatoires $(X_i)_i$ telle que 

$$ P(X_i+1|X_i) = P(X_i+1|X_i,\dots,X_1) $$

Autrement dit, la distribution de la prochaine variable ne dépend que de la variable actuelle. Une telle chaîne de Markov est dite "sans mémoire", car elle "oublie" le contexte antérieur.

La génération d'une suite de mots par une chaîne de Markov ne necessite alors que la matrice $P:(X_{i+1},X_i)\mapsto [0,1]$, appelée "matrice de transition".

On peut également relacher progressivement l'hypothèse forte de Markov en observant un contexte d'horizon fini ($k$ derniers mots). Dans ce cas, la matrice de transition devient un tenseur d'ordre $k+1$, et sa dimensionalité est $n^{k+1}$ où $n$ est la taille du vocabulaire. Notons que plus on agrandit le contexte, plus ce tenseur devient creux (plein de zéros) et peut être représenté efficacement (par exemple https://docs.scipy.org/doc/scipy/reference/sparse.html)


Dans ce TP nous allons explorer les capacités d'une IA markovienne qui :
- rédige automatiquement des SMS d'amour
- complète automatiquement les SMS d'amour que l'humain rédige


In [ ]:
# Affichage joli
def ecrit(mot):
    global point
    mot = mot.capitalize() if point else mot.lower()
    if mot not in '.,?!':
        point = mot in '.?!'
        print(" ", end='')
    else:
        point = True
    print(mot, end='')
    

In [ ]:
# Generation via la probabilité marginale
import numpy as np

with open("sms.txt", "r") as f:
    texte = f.read().split("\n")

index = {}
for w in texte:
    index[w] = index.get(w, 0) + 1
    
    
p = np.array(list(index.values()))
p = p / sum(p)

point = False
for i in range(1000):
    mot = np.random.choice(list(index.keys()), p=p)
    ecrit(mot)


In [ ]:
{k: v for k, v in sorted(index.items(), key=lambda x: x[1], reverse=True)}

In [ ]:
# C'est pas de la grande litterature...

# Generation via une chaine de Markov "stricte"

n = len(index)
M = np.zeros((n,n))
rev = {w:i for i,w in enumerate(index.keys())}

for a,b in zip(texte,texte[1:]):
    M[rev[a],rev[b]] += 1    
    
mot = '.'
point = False
for i in range(1000):
    mot = np.random.choice(list(index.keys()), p=M[rev[mot]]/sum(M[rev[mot]]))
    ecrit(mot)


In [ ]:
# On se sent déjà un peu mieux courtisé !

# Generation via une chaine de Markov avec un contexte de longueur 2

import re

n = len(index)
M = {}
rev = {w:i for i,w in enumerate(index.keys())}

def cle(a,b):
    def encode(x):
        return re.sub(r'[^\w]', 'SP', x.lower().replace('.','POINT').replace(',',"VIRGULE").replace('?', 'QUEST'))
    return (encode(a),encode(b))

for a,b,c in zip(texte,texte[1:],texte[2:]):
    if cle(a,b) not in M:
        M[cle(a,b)] = np.zeros(n)
    M[cle(a,b)][rev[c]] += 1    

a = '.'
b = '.'
point = False
for i in range(1000):
    mot = np.random.choice(list(index.keys()), p=M[cle(a,b)]/sum(M[cle(a,b)]))
    ecrit(mot)
    a = b
    b = mot
    

In [ ]:
# Generation via une chaine de Markov avec un contexte de longueur k donné

import re

K = 1

n = len(index)
M = {}
rev = {w:i for i,w in enumerate(index.keys())}

def cle(x):
    def encode(x):
        return x.lower().replace('.','POINT').replace(',',"VIRGULE").replace('?', 'QUEST')
    return tuple([encode(x) for x in x])

for i,w in enumerate(texte[:-K]):
    contexte = [ texte[i+j] for j in range(K) ]
    mot = texte[i+K]
    c = cle(contexte)
    if c not in M:
        M[c] = np.zeros(n)
    M[c][rev[mot]] += 1    

contexte = list(M.keys())[np.random.randint(len(M.keys()))]
point = False
for i in range(1000):
    c = cle(contexte)
    mot = np.random.choice(list(index.keys()), p=M[c]/sum(M[c]))
    _mot = mot.capitalize() if point else mot.lower()
    if mot not in '.,?!':
        point = mot in '.?!'
        print(" ", end='')
    else:
        point = True
    print(_mot, end='')
    contexte = contexte[1:] + (mot,)

> A vous de jouer : testez avec un contexte de longueur $k$ arbitraire. 

> Que se passe-t-il lorsqu'on augment trop $k$ ?

In [ ]:
import ipywidgets as widgets
from IPython.display import display

S = 3

t = widgets.Textarea(
    value='',
    placeholder="Tapez votre SMS d'amour assisté",
    description="Mon SMS d'amout assisté:",
    disabled=False
)
suggestions = [widgets.Button(description="") for _ in range(S)]

def proposer_mots(contexte, nb_suggestions):
    c = cle(contexte)
    mots = np.random.choice(list(index.keys()), p=M[c]/sum(M[c]), size=nb_suggestions)
    return mots

def on_change(change):
    try:
        mots = t.value.strip().replace(".", "").replace(",", "").lower().split(" ")
        contexte = mots[-K:]
        s = proposer_mots(contexte, S)
        for btn, sug in zip(suggestions, s):
            btn.description = sug
    except:
        for btn in suggestions:
            btn.description = ""
t.observe(on_change)

def accepter_suggestion(x):
    t.value += " " + x.description

display(t)
for s in suggestions:
    s.on_click(accepter_suggestion)
    display(s)

